# Build Initial Dataset 

Example of how to build the initial investing dataset


### 0. Importing Necessary Libs

In [2]:
import Extractors.Extract as methods
from pathlib import Path
import pandas as pd 
import json 

data_path = "./DATA"
path = Path(data_path)

### 1. First Build Fundamentus data

In [1]:
methods.build_fundamentos(data_path)

In [2]:
fundamentos = pd.read_csv(Path(data_path) / "fundamentos.csv")

In [3]:
assets = methods.build_full_assets(data_path, fundamentos)

In [4]:
grouping = assets.groupby('Codigo')['P/L'].sum().reset_index()
not_found_codes = grouping[grouping['P/L'] == 0]['Codigo']

In [5]:
teste = methods.get_fundamentos_simple()
teste['Codigo'] = teste.index.str[:4]
not_found_tickers = teste[teste['Codigo'].isin(not_found_codes)].index
not_found_codes = [x for x in not_found_codes if x not in teste['Codigo'].ravel()]

In [6]:
new_fundamentos = methods.build_fundamentos_tickers(data_path, not_found_tickers)

  0%|          | 0/65 [00:00<?, ?it/s]Building Fundamentus data

  2%|▏         | 1/65 [00:03<04:10,  3.92s/it]
error in trying to get data from ticker NTCO3
  5%|▍         | 3/65 [00:27<09:07,  8.83s/it]
error in trying to get data from ticker CEAB3
  9%|▉         | 6/65 [01:00<09:07,  9.29s/it]
error in trying to get data from ticker COGN3
 12%|█▏        | 8/65 [01:15<07:38,  8.05s/it]
error in trying to get data from ticker GPIV33
 14%|█▍        | 9/65 [01:19<06:18,  6.75s/it]
error in trying to get data from ticker ATMP3
 18%|█▊        | 12/65 [01:33<04:37,  5.23s/it]
error in trying to get data from ticker AVLL3
 20%|██        | 13/65 [01:37<04:11,  4.84s/it]
error in trying to get data from ticker CASH3
 22%|██▏       | 14/65 [01:40<03:49,  4.50s/it]
error in trying to get data from ticker CURY3
 23%|██▎       | 15/65 [01:44<03:38,  4.38s/it]
error in trying to get data from ticker DMVF3
 25%|██▍       | 16/65 [01:48<03:24,  4.17s/it]
error in trying to get data from ticker ENJU3

In [7]:
fundamentos = fundamentos.append(new_fundamentos)
fundamentos.to_csv(Path(data_path) / "b3" / "fundamentos.csv", index = False)
assets = methods.build_full_assets(data_path, fundamentos)

In [9]:
with open(Path(data_path) / 'log.txt', 'r') as f:
    log = f.readlines()
log = [x.replace("\n", "") for x in log]

In [49]:
renaming = {"Cotacao" : "Última Cotação ON",
"P/L" : "P/L",
"P/VP" : "P/VPA",
"PSR" : "PSR",
"DY" : "DY",
"P/Ativo" : "P/Ativo",
"P/Cap.Giro" : 'P/Capital de Giro',
"P/EBIT" : 'P/EBIT',
"P/ACL" : 'P/ACL',
"EV/EBIT" : 'EV/EBIT',
"EV/EBITDA" : "REMOVER",
"Mrg.Ebit" : 'Margem EBIT',
"Mrg.Liq." : "REMOVER",
"Liq.Corr." : "REMOVER",
"ROIC" : "REMOVER",
"ROE" : "ROE",
"Liq.2meses" : "REMOVER",
"Pat.Liq" : "REMOVER",
"Div.Brut/Pat." : "REMOVER",
"Cresc.5anos" : "REMOVER",
"Codigo" : "Codigo" }

In [48]:
teste['Ticker'] = teste.index
teste = teste.reset_index()
teste = teste[teste['Ticker'].isin(log)]
teste = teste.rename(columns = renaming)
teste = teste.drop(columns = "REMOVER")
teste = teste.drop(columns = 'index')
teste['Data'] = ["2020"] * len(teste)

In [68]:
fundamentos = fundamentos.append(teste)
fundamentos.to_csv(Path(data_path) / "b3" / "fundamentos.csv", index = False)
assets = methods.build_full_assets(data_path, fundamentos)

In [45]:
df = pd.read_csv(Path(data_path) / "b3" / "fundamentos.csv")

In [46]:
df = df.rename(columns = {'Data' : 'DATE', 'Ticker': "TICKER"})
df.to_csv(Path(data_path) / "b3" / "fundamentos.csv", index = False)

### 2. Extract Macro Price data 

Extracting 2 years of data for every ticker in assets database. This data will be daily. This will be used only for extracting some metrics. More precise data will be extract later to use in ML models.

In [5]:
assets = pd.read_csv(path / "b3" / "assets.csv")
tickers = assets['Ticker'].unique()

In [6]:
history = methods.build_price_history(data_path, tickers)

In [7]:
history = pd.read_csv(path / "b3" / "history.csv")

In [43]:
df = pd.read_csv(path / "b3" / "history.csv")

In [44]:
df = df.rename(columns = {'date' : 'DATE', 'symbol': "TICKER"})
df.to_csv(path / "b3" / "history.csv", index = False)

all_tickers = df["TICKER"].unique().ravel().tolist()
config = {'TICKERS' : all_tickers}
with open(path / "b3" / "config.json", 'w') as f:
    json.dump(config, f)

### Updating b3 history

In [3]:
import Extractors.b3.b3 as b3 
import json
with open(path / 'b3_history' / 'config.json', 'r') as f:
    config = json.load(f)

tickers = config['TICKERS']
api = b3.B3()

dfs = []
for ticker in tickers:
    try:
        data = api.Extract_History(ticker).reset_index().rename(columns = {"symbol" : "TICKER", "date" : "DATE"})
        dfs.append(data)
    except:
        print('\n could not load data from', ticker)

result = pd.concat(dfs, ignore_index = True)


 could not load data from PMAM3.SA

 could not load data from FHER3.SA

 could not load data from IDVL3.SA

 could not load data from IDVL4.SA


In [21]:
result['DATE'] = pd.to_datetime(result['DATE'])

In [25]:
for year, df_y in result.groupby(pd.Grouper(key = 'DATE', freq='Y')):
    year_tag = year.strftime('%Y')
    file_name = year_tag + '_data.parquet'
    original_data = pd.read_parquet(path / 'b3_history' / file_name)
    original_data['DATE'] = pd.to_datetime(original_data['DATE'])
    
    data = original_data.append(df_y)
    data = data.drop_duplicates(subset = ['TICKER', 'DATE'], keep = 'last')

    data.to_parquet(path / 'b3_history' / file_name)

### 3. Extract Cripto data 

Extracting 2 years of data for every ticker in assets database. This data will be daily. This will be used only for extracting some metrics. More precise data will be extract later to use in ML models.

In [2]:
criptos = pd.read_csv(Path("./Extractors/cripto/digital_currency_list.csv"))

In [3]:
history = methods.build_cripto_history(data_path / "cripto", criptos['currency code'].unique().ravel())

 history from ELEC 

 43%|████▎     | 234/542 [02:51<01:40,  3.08it/s]Could not read history from ELIX 

 43%|████▎     | 235/542 [02:52<01:31,  3.34it/s]Could not read history from EMB 

 44%|████▎     | 236/542 [02:52<01:29,  3.41it/s]Could not read history from EMC 

 44%|████▍     | 238/542 [02:53<02:15,  2.24it/s]Could not read history from ENG 

 44%|████▍     | 239/542 [02:53<02:00,  2.52it/s]Could not read history from ENRG 

 44%|████▍     | 240/542 [02:54<01:46,  2.83it/s]Could not read history from EOT 

 44%|████▍     | 241/542 [02:54<01:38,  3.05it/s]Could not read history from EQT 

 45%|████▍     | 242/542 [02:54<01:34,  3.18it/s]Could not read history from ERC 

 45%|████▍     | 243/542 [02:54<01:28,  3.39it/s]Could not read history from ETHD 

 45%|████▌     | 246/542 [02:58<03:20,  1.47it/s]Could not read history from ETT 

 46%|████▌     | 247/542 [02:58<02:46,  1.77it/s]Could not read history from EVE 

 46%|████▌     | 248/542 [02:58<02:23,  2.05it/s]Could not read

In [39]:
df = pd.read_csv(path / "cripto" / "cripto_history.csv")

In [42]:
df = df.rename(columns = {'date' : 'DATE', 'symbol': "TICKER"})
df.to_csv(path / "cripto" / "cripto_history.csv", index = False)

all_tickers = df["TICKER"].unique().ravel().tolist()
config = {'TICKERS' : all_tickers}
with open(path / "cripto" / "config.json", 'w') as f:
    json.dump(config, f)

### 4. Extract Fundos data 

Extracting data from fundos de investimento

In [2]:
fundos = methods.build_fundos_history(data_path / "funds")

 44%|████▍     | 11/25 [01:17<04:36, 19.78s/it]Could not get data for 201912 

100%|██████████| 25/25 [01:48<00:00,  4.34s/it]


In [51]:
df = pd.read_csv(path / "funds" /  "fundos_history.csv")

In [52]:
df = df.rename(columns = {'DT_COMPTC' : 'DATE', 'CNPJ_FUNDO': "TICKER"})
df.to_csv(path / "funds" / "fundos_history.csv", index = False)

all_tickers = df["TICKER"].unique().ravel().tolist()
config = {'TICKERS' : all_tickers}
with open(path / "funds" / "config.json", 'w') as f:
    json.dump(config, f)

### 5. Extract Other Assets

FIIs, ETFs, BDRs

In [2]:
history = methods.build_fundos_b3_history(data_path / "b3_funds")

  0%|          | 0/1030 [00:00<?, ?it/s]Could not read history from AFHI11 

  0%|          | 2/1030 [00:02<19:22,  1.13s/it]Could not read history from AFOF11 

  0%|          | 4/1030 [00:05<24:04,  1.41s/it]Could not read history from ARFI11 

  1%|          | 9/1030 [00:15<36:23,  2.14s/it]Could not read history from BBFI11 

  1%|          | 12/1030 [00:19<30:54,  1.82s/it]Could not read history from BBIM11 

  2%|▏         | 17/1030 [00:34<57:25,  3.40s/it]Could not read history from BLMC11 

  3%|▎         | 26/1030 [00:55<31:59,  1.91s/it]Could not read history from BICE11 

  3%|▎         | 30/1030 [01:01<28:59,  1.74s/it]Could not read history from BTWR11 

  3%|▎         | 31/1030 [01:02<25:26,  1.53s/it]Could not read history from BTSG11 

  4%|▎         | 37/1030 [01:12<29:52,  1.81s/it]Could not read history from CFHI11 

  4%|▍         | 45/1030 [01:27<32:16,  1.97s/it]Could not read history from DLMT11 

  5%|▍         | 47/1030 [01:30<27:58,  1.71s/it]Could not read hi

In [53]:
df = pd.read_csv(path / "b3_funds" / "fundos_b3_history.csv")

In [55]:
df = df.rename(columns = {'date' : 'DATE', 'symbol': "TICKER"})
df.to_csv(path / "b3_funds" / "fundos_b3_history.csv", index = False)

all_tickers = df["TICKER"].unique().ravel().tolist()
config = {'TICKERS' : all_tickers}
with open(path / "b3_funds" / "config.json", 'w') as f:
    json.dump(config, f)

In [56]:
df = pd.read_csv(path / "b3_funds" / "assets_fundos_b3.csv" )

In [58]:
df = df.rename(columns = {'Ticker': "TICKER"})
df.to_csv(path / "b3_funds" / "assets_fundos_b3.csv", index = False)

## Adjusting history to be year file

In [6]:
import pyarrow as pa
import pyarrow.parquet as pq

In [13]:
asset = 'funds_history'
df = pd.read_csv(path / asset / 'fundos_history.csv', parse_dates=['DATE'])

In [14]:
timestamp = df['DATE'].dt.strftime("%Y")
for t, dfg in df.groupby(timestamp):    
    _table = pa.Table.from_pandas(dfg)
    file_name = t + "_data.parquet"
    pq.write_table(_table, path / asset / file_name)

## Get KPIs 

In [29]:
import pandas as pd 
from Extractors.indicadores import bcb
from pathlib import Path 
from Extractors import b3
from datetime import datetime
from time import sleep
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq

In [24]:
b3_api = b3.b3.B3()
dir = Path('./DATA')

In [25]:
def random_wait():
    wait_times = [0.2, 0.5, 1, 2, 4]
    probs = [0.3, 0.4, 0.2, 0.08, 0.02]
    choice = np.random.choice(wait_times, size=1, p=probs)
    sleep(choice[0])

def extract_intraday(asset, tickers):    
    all_data = pd.DataFrame([])
    errors = []
    for ticker in tickers:
        try:
            data = b3_api.Extract_History(ticker)
            data = data.reset_index().rename(columns = {"symbol" : "TICKER", "date" : "DATE"})
            all_data = all_data.append(data)
        except Exception as e: 
            print("Could not load data from", ticker, "\n")
            print(e)
            errors.append(ticker)
        random_wait()

    log_file = datetime.now().strftime("%Y%m%d") +"_log.txt"
    with open(dir / asset / log_file, 'w') as f:
        f.write('\n'.join(errors))

    return all_data

In [26]:
data = []
for ticker in ['selic', 'ipca', 'igpm', 'cdi', 'pnad', 'cambio', 'pib']:
    ds = bcb.get_data_bcb(ticker, start = "2000-01-01", end = "2021-02-25")
    ds = ds.rename(columns = {'data' : 'DATE', 'valor' : 'close'})
    ds['TICKER'] = [ticker] * len(ds)
    data.append(ds)

data.append(extract_intraday('KPIs_history', ["^BVSP", 'IFIX.SA']))

result = pd.concat(data, ignore_index = True)

In [37]:
df = result
asset = 'KPIs_history'
df['DATE'] = pd.to_datetime(df['DATE'])

In [38]:

timestamp = df['DATE'].dt.strftime("%Y")
for t, dfg in df.groupby(timestamp):    
    _table = pa.Table.from_pandas(dfg)
    file_name = t + "_data.parquet"
    pq.write_table(_table, path / asset / file_name)